In [2]:
import os
import glob
import pandas as pd
import numpy as np
import json
from sklearn.utils import shuffle
from keras.preprocessing import sequence, image
from keras.preprocessing.image import array_to_img, save_img, img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from keras import metrics
from keras.layers import (
    Flatten,
    Dense,
    Input,
    Activation,
    BatchNormalization,
    Conv2D,
    MaxPool2D,
    Dropout,
    UpSampling2D,
    Lambda,
)

import pickle
from keras.layers import ReLU, Reshape, Conv2DTranspose, Concatenate, Multiply
from keras.models import Model

from tensorflow.keras.optimizers import Adam
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras import backend as K

from keras.callbacks import ModelCheckpoint, EarlyStopping
from collections import Counter
from keras import metrics
from notebook_utils import initialize_environment
initialize_environment()

from utils.data_loading.load_data import get_tile_data

from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval
import tensorflow as tf

In [3]:
data_directory = "../data/context_data/"
json_directory = "../data/json_files_trimmed_features/"

game_data_path="../data/game_data.csv"
train_data_path="../data/train_data.csv"
test_data_path="../data/test_data.csv"

EPOCHS=10
BATCH_SIZE=25
LATENT_DIM=256

In [4]:
def loss_func_image(y_true, y_pred):
    # tile sprite loss
    r_loss=K.mean(K.square(y_true - y_pred), axis=[1,2,3])
    loss  =  r_loss
    return loss
    
def loss_func_text(y_true,y_pred):
    # multilabel text weighted bce
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    bce_array=-(y_true*K.log(y_pred)+(1-y_true)*K.log(1-y_pred))
    weighted_array=bce_array*tensor_from_list
    bce_sum=K.sum(weighted_array,axis=1)
    loss=bce_sum/13.0
    return loss

def check_nonzero(y_true,y_pred):
    """
    Custom metric
    Returns sum of all embeddings
    """
    return(K.sum(K.cast(y_pred > 0.4, 'int32')))

def get_pickle_file(path):
    with open(path,"rb") as handle:
        return pickle.load(handle)

In [5]:
# data loading
data=pd.read_csv(game_data_path)
train_data=pd.read_csv(train_data_path)
test_data=pd.read_csv(test_data_path)

data['features'] = data.features.apply(lambda x: literal_eval(str(x)))
train_data['features'] = train_data.features.apply(lambda x: literal_eval(str(x)))
test_data['features'] = test_data.features.apply(lambda x: literal_eval(str(x)))

# loading multi-label binarizer
mlb=get_pickle_file("../model/model_tokenizer.pickle")
print("Feature Dictionary Loaded")
total_features = len(mlb.classes_)
print("The feature dictionary has size", total_features)
display("Features", mlb.classes_)

# loading the batches
# training 
train_image_batch=get_pickle_file("../data/train_image_batch.pickle")
train_text_batch=get_pickle_file("../data/train_text_batch.pickle")
output_image_batch=get_pickle_file("../data/output_image_batch.pickle")
output_text_batch=get_pickle_file("../data/output_text_batch.pickle")

#testing
test_image_batch=get_pickle_file("../data/test_image_batch.pickle")
test_text_batch=get_pickle_file("../data/test_text_batch.pickle")

print("\Training Testing Batches loaded")

print("Train Image batch shape", train_image_batch.shape)
print("Train Text batch shape", train_text_batch.shape)
print("Train Output Image batch shape", output_image_batch.shape)
print("Train Output Text batch shape", output_text_batch.shape)

print("Test Image batch shape", test_image_batch.shape)
print("Test Text batch shape", test_text_batch.shape)   

Feature Dictionary Loaded
The feature dictionary has size 13


'Features'

array(['block', 'breakable', 'climbable', 'collectable', 'element',
       'empty', 'hazard', 'moving', 'openable', 'passable', 'pipe',
       'solid', 'wall'], dtype=object)

\Training Testing Batches loaded
Train Image batch shape (18494, 48, 48, 3)
Train Text batch shape (18494, 13)
Train Output Image batch shape (18494, 16, 16, 3)
Train Output Text batch shape (18494, 13)
Test Image batch shape (2055, 48, 48, 3)
Test Text batch shape (2055, 13)


In [6]:
#initialise the TF-IDF vectorizer to counter imbalanced dataset
vectorizer = TfidfVectorizer(stop_words=None)
train_data_copy = train_data
train_data_copy["features"] = train_data_copy.features.apply(lambda x: str(x))
vectors = vectorizer.fit_transform(train_data_copy["features"])
idf = vectorizer.idf_
new_dict = {}
for c in mlb.classes_:
    if c in vectorizer.vocabulary_.keys():
        new_dict[c] = idf[vectorizer.vocabulary_[c]]
    else:
        new_dict[c] = np.max(idf)
print("\n Printing the TF-IDF for the labels\n\n", new_dict)
weight_freq = {k: v / sum(new_dict.values()) for k, v in new_dict.items()}
print("\nPrinting the weight normalised\n\n")
print(weight_freq)
weight_vector = [v * 1000 for v in new_dict.values()]
tensor_from_list = tf.convert_to_tensor(weight_vector)
tensor_from_list = K.cast(tensor_from_list, "float32")
print("Weight Vector")
print(weight_vector)


 Printing the TF-IDF for the labels

 {'block': 4.895666560876659, 'breakable': 2.527712174910271, 'climbable': 2.7397689321637086, 'collectable': 4.067161199838823, 'element': 5.357195563131423, 'empty': 2.0458521063722053, 'hazard': 4.6853711520402985, 'moving': 5.965443299904882, 'openable': 6.220085518278463, 'passable': 1.5640321219758018, 'pipe': 6.190526716036919, 'solid': 1.8879058559991626, 'wall': 5.050704158722146}

Printing the weight normalised


{'block': 0.09202826128752296, 'breakable': 0.04751568629106969, 'climbable': 0.051501908477902945, 'collectable': 0.07645409852631353, 'element': 0.10070403834119256, 'empty': 0.038457727841484414, 'hazard': 0.08807514875600009, 'moving': 0.11213781981942068, 'openable': 0.11692455934016166, 'passable': 0.029400522889674464, 'pipe': 0.11636891586603913, 'solid': 0.03548866967178095, 'wall': 0.09494264289143667}
Metal device set to: Apple M1
Weight Vector
[4895.666560876659, 2527.7121749102707, 2739.7689321637085, 4067.1611998388

2021-09-09 07:31:19.752736: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-09-09 07:31:19.752840: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# model definition
latent_dim = 128
batch_size = 1
# image encoder
image_encoder_input = Input(shape=(48, 48, 3), name="image_input")
image_encoder_conv_layer1 = Conv2D(
    32, strides=3, kernel_size=(3, 3), name="iencode_conv1"
)(image_encoder_input)
image_encoder_norm_layer1 = BatchNormalization()(image_encoder_conv_layer1)
image_encoder_actv_layer1 = ReLU()(image_encoder_norm_layer1)
image_encoder_conv_layer2 = Conv2D(32, (3, 3), padding="same", name="iencode_conv2")(
    image_encoder_actv_layer1
)
image_encoder_norm_layer2 = BatchNormalization()(image_encoder_conv_layer2)
image_encoder_actv_layer2 = ReLU()(image_encoder_norm_layer2)
image_encoder_conv_layer3 = Conv2D(16, (3, 3), padding="same", name="iencode_conv3")(
    image_encoder_actv_layer2
)
image_encoder_norm_layer3 = BatchNormalization()(image_encoder_conv_layer3)
image_encoder_actv_layer3 = ReLU()(image_encoder_norm_layer3)
image_shape_before_flatten = K.int_shape(image_encoder_actv_layer3)[1:]
image_flatten = Flatten(name="image_flatten_layer")(image_encoder_actv_layer3)

# text encoder
text_encoder_input = Input(shape=(13,))
text_encoder_dense_layer1 = Dense(32, activation="tanh", name="tencode_dense1")(
    text_encoder_input
)
text_encoder_dense_layer2 = Dense(16, activation="tanh", name="tencode_dense2")(
    text_encoder_dense_layer1
)
text_shape_before_concat = K.int_shape(text_encoder_dense_layer2)[1:]

# image-text concatenation
image_text_concat = Concatenate(name="image_text_concatenation")(
    [image_flatten, text_encoder_dense_layer2]
)
image_text_concat = Dense(256, activation="tanh", name="embedding_dense_1")(
    image_text_concat
)


# define encoder model
encoding_model = Model(
    inputs=[image_encoder_input, text_encoder_input], outputs=image_text_concat
)

# decoder for image
# decoder_input=Input(shape=(512,))
image_y = Dense(units=np.prod(image_shape_before_flatten), name="image_dense")(
    image_text_concat
)
image_y = Reshape(target_shape=image_shape_before_flatten, name="image_reshape")(
    image_y
)
image_decoder_convt_layer1 = Conv2DTranspose(
    16, (3, 3), padding="same", name="idecode_conv1"
)(image_y)
image_decoder_norm_layer1 = BatchNormalization(name="idecode_norm1")(
    image_decoder_convt_layer1
)
image_decoder_actv_layer1 = ReLU(name="idecode_relu1")(image_decoder_norm_layer1)
image_decoder_convt_layer2 = Conv2DTranspose(
    32, (3, 3), padding="same", name="idecode_conv2"
)(image_decoder_actv_layer1)
image_decoder_norm_layer2 = BatchNormalization(name="idecode_norm2")(
    image_decoder_convt_layer2
)
image_decoder_actv_layer2 = ReLU(name="idecode_relu2")(image_decoder_norm_layer2)
image_decoder_output = Conv2DTranspose(
    3, (3, 3), padding="same", name="image_output_layer"
)(image_decoder_actv_layer2)


# decoder for text
text_decoder_dense_layer1 = Dense(16, activation="tanh", name="tdecode_dense1")(
    image_text_concat
)
text_reshape = Reshape(target_shape=text_shape_before_concat, name="text_reshape")(
    text_decoder_dense_layer1
)
text_decoder_dense_layer2 = Dense(32, activation="tanh", name="tdecode_dense2")(
    text_reshape
)
text_decoder_output = Dense(13, activation="sigmoid", name="text_output_layer")(
    text_decoder_dense_layer2
)
# decoding_model=Model(inputs=[decoder_input],outputs=[image_decoder_output,text_decoder_output])

ae_sep_output = Model(
    [image_encoder_input, text_encoder_input],
    [image_decoder_output, text_decoder_output],
)

In [8]:
losses ={'image_output_layer':loss_func_image,
          'text_output_layer':loss_func_text,
}
#tweak loss weights
lossWeights={'image_output_layer':0.1,
          'text_output_layer':0.9  
        }


accuracy={
    'image_output_layer':loss_func_image,
    'text_output_layer': check_nonzero
}

ae_sep_output.compile(
    optimizer="adam", loss=losses, loss_weights=lossWeights, metrics=accuracy
)

es = EarlyStopping(
    monitor="val_text_output_layer_loss", mode="min", verbose=1, patience=2
)

ae_history = ae_sep_output.fit(
    [train_image_batch, train_text_batch],
    [output_image_batch, output_text_batch],
    epochs=10,
    batch_size=25,
    shuffle=True,
    validation_split=0.2,
    callbacks=[es],
)

2021-09-09 07:31:28.619179: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-09 07:31:28.619410: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


2021-09-09 07:31:37.838757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


592/592 [==============================] - ETA: 0s - loss: 1689.8965 - image_output_layer_loss: 8858.6509 - text_output_layer_loss: 893.3685 - image_output_layer_loss_func_image: 8858.6512 - text_output_layer_check_nonzero: 59.9411

2021-09-09 07:31:48.113921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


592/592 [==============================] - 21s 19ms/step - loss: 1689.0060 - image_output_layer_loss: 8854.0172 - text_output_layer_loss: 892.8939 - image_output_layer_loss_func_image: 8854.0175 - text_output_layer_check_nonzero: 59.9233 - val_loss: 569.9338 - val_image_output_layer_loss: 2852.4338 - val_text_output_layer_loss: 316.3228 - val_image_output_layer_loss_func_image: 2852.4338 - val_text_output_layer_check_nonzero: 44.2432
Epoch 2/10
592/592 [==============================] - 11s 18ms/step - loss: 451.3461 - image_output_layer_loss: 2359.4762 - text_output_layer_loss: 239.3315 - image_output_layer_loss_func_image: 2359.4762 - text_output_layer_check_nonzero: 44.3932 - val_loss: 275.3939 - val_image_output_layer_loss: 1574.3854 - val_text_output_layer_loss: 131.0616 - val_image_output_layer_loss_func_image: 1574.3855 - val_text_output_layer_check_nonzero: 46.2297
Epoch 3/10
592/592 [==============================] - 11s 18ms/step - loss: 263.1439 - image_output_layer_loss: 16

In [9]:
ae_history = ae_sep_output.fit(
    [train_image_batch, train_text_batch],
    [output_image_batch, output_text_batch],
    epochs=5,
    batch_size=25,
    shuffle=True,
    validation_split=0.2,
    callbacks=[es],
)

Epoch 1/5
592/592 [==============================] - 11s 18ms/step - loss: 51.9090 - image_output_layer_loss: 465.7723 - text_output_layer_loss: 5.9241 - image_output_layer_loss_func_image: 465.7723 - text_output_layer_check_nonzero: 46.6723 - val_loss: 49.7016 - val_image_output_layer_loss: 436.4905 - val_text_output_layer_loss: 6.7251 - val_image_output_layer_loss_func_image: 436.4905 - val_text_output_layer_check_nonzero: 46.5473
Epoch 2/5
592/592 [==============================] - 11s 18ms/step - loss: 47.1144 - image_output_layer_loss: 423.9874 - text_output_layer_loss: 5.2397 - image_output_layer_loss_func_image: 423.9874 - text_output_layer_check_nonzero: 46.6723 - val_loss: 55.2789 - val_image_output_layer_loss: 441.1127 - val_text_output_layer_loss: 12.4084 - val_image_output_layer_loss_func_image: 441.1127 - val_text_output_layer_check_nonzero: 46.4392
Epoch 3/5
592/592 [==============================] - 11s 18ms/step - loss: 55.8166 - image_output_layer_loss: 480.3064 - text

In [10]:
# build the inference model

decoder_input = Input(shape=(256,))

d_dense = ae_sep_output.get_layer("image_dense")(decoder_input)
d_reshape = ae_sep_output.get_layer("image_reshape")(d_dense)
d_conv1 = ae_sep_output.get_layer("idecode_conv1")(d_reshape)
d_norm1 = ae_sep_output.get_layer("idecode_norm1")(d_conv1)
d_relu1 = ae_sep_output.get_layer("idecode_relu1")(d_norm1)
d_conv2 = ae_sep_output.get_layer("idecode_conv2")(d_relu1)
d_norm2 = ae_sep_output.get_layer("idecode_norm2")(d_conv2)
d_relu2 = ae_sep_output.get_layer("idecode_relu2")(d_norm2)
d_image_output = ae_sep_output.get_layer("image_output_layer")(d_relu2)

t_dense = ae_sep_output.get_layer("tdecode_dense1")(decoder_input)
t_reshape = ae_sep_output.get_layer("text_reshape")(t_dense)
t_dense2 = ae_sep_output.get_layer("tdecode_dense2")(t_reshape)
d_text_output = ae_sep_output.get_layer("text_output_layer")(t_dense2)

decoder_model = Model(inputs=[decoder_input], outputs=[d_image_output, d_text_output])

In [11]:
# saving the entire architecture model
model_json = ae_sep_output.to_json()
with open("../model/autoencoder_model_test.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
ae_sep_output.save_weights("../model/autoencoder_model_test.h5")
print("Saved Entire Model to disk")

# saving the encoder part
model_json = encoding_model.to_json()
with open("../model/encoder_model_test.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
encoding_model.save_weights("../model/encoder_model_test.h5")
print("Saved Encoder Model to disk")
# saving the encoder part

model_json = decoder_model.to_json()
with open("../model/decoder_model_test.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
decoder_model.save_weights("../model/decoder_model_test.h5")
print("Saved Decoder Model to disk")

Saved Entire Model to disk
Saved Encoder Model to disk
Saved Decoder Model to disk
